In [6]:
import pennylane as qml
import numpy as np

# The following cells are the helper functions and their tests

I will use Basis Embedding and Grover algorithm to build the final function find_primes_numbers

In [2]:
def adder(lista, listb, listc):
    '''
    a full adder
    lista: list of qubits of number A, len=n
    listb: list of qubits of number B, len=n
    listc: list of qubits of carry bits, len=n, will store the result
    '''
    n = len(lista)
    for i in range(n-1):
        qml.Toffoli([lista[n-1-i], listc[n-1-i], listc[n-2-i]])
        qml.CNOT([lista[n-1-i], listc[n-1-i]])
        qml.Toffoli([listc[n-1-i], listb[n-1-i], listc[n-2-i]])
        qml.CNOT([listb[n-1-i], listc[n-1-i]])
    qml.CNOT([lista[0], listc[0]])
    qml.CNOT([listb[0], listc[0]])

In [9]:
# test adder
dev = qml.device('default.qubit', wires=6, shots=100 )
@qml.qnode(dev)
def test_add():
    qml.PauliX(0)
    qml.PauliX(1)
    qml.PauliX(2)
    adder([0,1],[2,3],[4,5])
    qml.adjoint(adder)([0,1],[2,3],[4,5])
    return qml.counts()
test_add()

{'111000': tensor(100, requires_grad=True)}

In [3]:
def if_sum(qA, qB, qN, auxiliary, listc):
    '''
    a function that:
    if numA+numB = numN: auxiliary return state 1
    if numA+numB != numN: auxiliary return state 0
    
    qA: list of wires that encoded number A, len=n, keep qA state non change
    qB: list of wires that encoded number B, len=n, keep qB state non change
    qN: list of wires that encoded number N, len=n, keep qN state non change
    listc: list of wires of carry bits when sum numA, numB, len=n, keep listc state non change
    auxiliary: auxiliary qubit
    
    '''
    n = len(qA)
    adder(qA, qB, listc)
    for i in range(n):
        qml.CNOT([qN[i], listc[i]])
        qml.PauliX(listc[i])
        
    qml.MultiControlledX(listc,auxiliary)
    
    for ii in range(n):
        qml.PauliX(listc[ii])
        qml.CNOT([qN[ii], listc[ii]])
    qml.adjoint(adder)(qA, qB, listc)
        

In [6]:
# test if_sum
dev = qml.device('default.qubit', wires=9)
@qml.qnode(dev)
def test_if_sum():
    qA=[0,1]
    qB=[2,3]
    qN=[4,5]
    listc=[6,7]
    auxiliary=[8]
    #qml.BasisEmbedding(1,qA)
    qml.Hadamard(0)
    qml.Hadamard(1)
    qml.BasisEmbedding(2,qB)
    qml.BasisEmbedding(3,qN)
    if_sum(qA, qB, qN, auxiliary, listc)
    return qml.expval(qml.PauliZ(6)),qml.expval(qml.PauliZ(7))
test_if_sum()

(tensor(1., requires_grad=True), tensor(1., requires_grad=True))

In [4]:
def encode_superposition(list_of_numA, qA):
    '''
    use Basis Embedding to encode the numbers in list_of_numA
    as a superposition on wires qA
    '''
    
    state_vector=np.zeros(2**len(qA))
    for i in list_of_numA:
        state_vector[i]=1.0/np.sqrt(len(list_of_numA))
    qml.StatePrep(state_vector, wires=qA)

In [5]:
#test encode_superposition
dev = qml.device('default.qubit', wires=5, shots=100 )
@qml.qnode(dev)
def test_encode():
    encode_superposition([1,3,7], [0,1,2,3,4])
    return qml.counts()
test_encode()

{'00001': 35, '00011': 30, '00111': 35}

In [5]:
def Gover_diffusion(list_of_num, qA, qB):
    '''
    diffusion operator for Gover search on both qA and qB
    qA: list of wires that encoded numA
    qB: list of wires that encoded numB

    the number of qubits n in qA and qB are the same, n=int(log2(numN))+1
    '''
    n = 2**len(qA)
    state_vector=np.zeros(n)
    for i in list_of_num:
        state_vector[i]=1.0/np.sqrt(len(list_of_num))
    op_a = np.outer(state_vector, state_vector)
    
    # tensor product
    op_ab = np.zeros((n**2,n**2))
    for j in range(n):
        for k in range(n):
            for l in range(n):
                for m in range(n):
                    op_ab[j*n+l][k*n+m] = op_a[j][k]*op_a[l][m]

    op_diffusion = op_ab*2-np.identity(n**2)
    qml.QubitUnitary(op_diffusion, wires=qA+qB)
    
    

In [56]:
# test diffusion
dev = qml.device('default.qubit', wires=4, shots=100 )
@qml.qnode(dev)
def test_diffu():
    qA=[0,1]
    qB=[2,3]
    encode_superposition([1,3], qA)
    encode_superposition([1,3], qB)
    Gover_diffusion([1,3], qA, qB)
    Gover_diffusion([1,3], qA, qB)
    
    return qml.counts()
test_diffu()

{'0101': 24, '0111': 21, '1101': 27, '1111': 28}

# The following cells are the finial function and the test 

In [132]:
def find_primes_numbers2(numN, list_of_num):
    '''
    Use Grover search with multiple solutions to find out primes numbers
    '''
    n = int(np.log2(numN))+1
    dev = qml.device('default.qubit', wires=4*n+1, shots=2)
    qA = [a for a in range(n)]
    qB = [b for b in range(n,2*n)]
    qN = [i for i in range(2*n,3*n)]
    listc = [c for c in range(3*n, 4*n)]
    auxiliary = 4*n
    

    m = int(np.sqrt(len(list_of_num))*np.pi/4) # repeat oracle and diffusion times

    @qml.qnode(dev) 
    def Gover_search(list_of_num, qA, numB, qB, qN, listc, auxiliary):       
        '''
        use Gover algoritm to find out which number in
        list_of_num saitisfies numA+numB=numN
        qA: list of wires to encode numA
        qB: list of wires to encode numB
        qN: list of wires to encode numN
        listc: list of wires of carry bits
        auxiliary: auxiliary qubit
        the number of qubits n in qA, qB and qN are the same, n=int(log2(numN))+1
        '''   
        encode_superposition(list_of_num, qA)
        qml.BasisEmbedding(numB, qB)
        qml.BasisEmbedding(numN, qN)
        qml.PauliX(auxiliary)
        qml.Hadamard(auxiliary)
        for i in range(m):
            if_sum(qA, qB, qN, auxiliary, listc)
            Gover_diffusion2(list_of_num, qA)
        return qml.counts(wires=qA)
    
    
    for bb in list_of_num:
        result = Gover_search(list_of_num, qA, bb, qB, qN, listc, auxiliary)
        dev.reset()
        if len(result) == 1:
            break
    
    str_a = list(result.keys())[0]
    A = int(str_a,2)
    B = bb
    return str(A)+","+str(B)
    

In [51]:
# test find_primes_numbers
find_primes_numbers(18, [1,3,5,7,11,13,15])

'5,13'

# Discussion
I used Grover algorithm with multiple solutions to solve this problem. Only one shot is needed to find out the primes numbers. However, Grover algorithm requires that the number of solutions is known in advance. To find out the number of solutions, we need hunders of shots.

I think Grover algorithm with multiple solutions does not show quantum advantages in this case.

To achive quantum advantages, I also tried using Grover algorithm with single solution for each number in the list.

# help functions

In [53]:
def Gover_diffusion2(list_of_num, qA):
    '''
    diffusion operator for Gover search on qA
    qA: list of wires that encoded numA

    the number of qubits n in qA and qB are the same, n=int(log2(numN))+1
    '''
    n = 2**len(qA)
    state_vector=np.zeros(n)
    for i in list_of_num:
        state_vector[i]=1.0/np.sqrt(len(list_of_num))
    op_a = np.outer(state_vector, state_vector)
    
    op_diffusion = op_a*2-np.identity(n)
    qml.QubitUnitary(op_diffusion, wires=qA)
    

# Final function and test

In [132]:
def find_primes_numbers2(numN, list_of_num):
    '''
    Use Grover search with multiple solutions to find out primes numbers
    '''
    n = int(np.log2(numN))+1
    dev = qml.device('default.qubit', wires=4*n+1, shots=2)
    qA = [a for a in range(n)]
    qB = [b for b in range(n,2*n)]
    qN = [i for i in range(2*n,3*n)]
    listc = [c for c in range(3*n, 4*n)]
    auxiliary = 4*n
    

    m = int(np.sqrt(len(list_of_num))*np.pi/4) # repeat oracle and diffusion times

    @qml.qnode(dev) 
    def Gover_search(list_of_num, qA, numB, qB, qN, listc, auxiliary):       
        '''
        use Gover algoritm to find out which number in
        list_of_num saitisfies numA+numB=numN
        qA: list of wires to encode numA
        qB: list of wires to encode numB
        qN: list of wires to encode numN
        listc: list of wires of carry bits
        auxiliary: auxiliary qubit
        the number of qubits n in qA, qB and qN are the same, n=int(log2(numN))+1
        '''   
        encode_superposition(list_of_num, qA)
        qml.BasisEmbedding(numB, qB)
        qml.BasisEmbedding(numN, qN)
        qml.PauliX(auxiliary)
        qml.Hadamard(auxiliary)
        for i in range(m):
            if_sum(qA, qB, qN, auxiliary, listc)
            Gover_diffusion2(list_of_num, qA)
        return qml.counts(wires=qA)
    
    
    for bb in list_of_num:
        result = Gover_search(list_of_num, qA, bb, qB, qN, listc, auxiliary)
        dev.reset()
        if len(result) == 1:
            break
    
    str_a = list(result.keys())[0]
    A = int(str_a,2)
    B = bb
    return str(A)+","+str(B)
    

In [142]:
# test find_primes_numbers2
find_primes_numbers2(18, [1,3,5,7,11,13,15])

'15,3'

# Discussion
The second method use only two shots for every Grover search which shows quantum advantage. This method works better when the length of the list is long, but if the lenth is short, the accuarcy of this method is low.